In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.circuit.library import RawFeatureVector

from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# ----------------------- 1. VQC Cuantic cu parametri -----------------------
class VQCCircuit:

    # Initializeaza un circuit cuantic cu 16 qubiti
    # Foloseste un circuit variational (TwoLocal) cu rotatii Ry si entanglement CZ
    def __init__(self, kernel_size, backend, shots=128):
        self.n_qubits = 16  # 2^4 pentru RawFeatureVector
        self.backend = backend
        self.shots = shots

        # Parametrii circuitului sunt initial random

        self.params = np.random.uniform(0, 2 * np.pi, size=self.n_qubits * 2)

        self.vqc = TwoLocal(num_qubits=self.n_qubits,
                            rotation_blocks='ry',
                            entanglement_blocks='cz',
                            reps=1,
                            parameter_prefix='θ')

    def run(self, data):

        # Normalizeaza patch-ul si il codifica in circuit
        # Ruleaza circuitul si returneaza o valoare scalara medie pe baza masuratorii
       
        data = data.flatten().astype(np.float64)

        if not np.all(np.isfinite(data)):
            return 0.0  # fallback safe value

        norm = np.linalg.norm(data)
        if norm == 0 or np.isnan(norm):
            return 0.0  # patch complet negru = output neutru

        data = data / norm

        if len(data) < self.n_qubits:
            data = np.pad(data, (0, self.n_qubits - len(data)))
        elif len(data) > self.n_qubits:
            data = data[:self.n_qubits]

        # Construim un feature map care ia vectorul normalizat si-l pune in stare cuantica

        feature_map = RawFeatureVector(self.n_qubits)
        feature_map = feature_map.assign_parameters(data)

        qc = QuantumCircuit(self.n_qubits)
        qc.compose(feature_map, inplace=True)

        vqc_bound = self.vqc.assign_parameters(self.params)
        qc.compose(vqc_bound, inplace=True)

        qc.measure_all()

        transpiled = transpile(qc, self.backend)
        job = self.backend.run(transpiled, shots=self.shots)
        result = job.result()
        counts = result.get_counts()

        total = sum(sum(int(bit) for bit in key) * val for key, val in counts.items())
        return total / (self.shots * self.n_qubits)

# -------------------------- 2. Funcția Forward -----------------------------
class QuanvFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, inputs, quantum_circuits, kernel_size):
         # Aplica circuitele cuantice pe fiecare patch din imagine
        batch_size = inputs.shape[0]
        length_x = inputs.shape[2] - kernel_size + 1
        length_y = inputs.shape[3] - kernel_size + 1
        outputs = torch.zeros(batch_size, len(quantum_circuits), length_x, length_y).to(inputs.device)

        for i in range(batch_size):
            for c, circuit in enumerate(quantum_circuits):
                for x in range(length_x):
                    for y in range(length_y):
                        patch = inputs[i, 0, x:x+kernel_size, y:y+kernel_size]
                        outputs[i, c, x, y] = circuit.run(patch.cpu().detach().numpy())

        return outputs

    @staticmethod
    def backward(ctx, grad_output):
        return None, None, None

# ---------------------------- 3. Stratul Cuantic ----------------------------
class QuanvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(QuanvLayer, self).__init__()
        # Creeaza un strat cuantic cu mai multe circuite (cate unul pe canal)
        backend = Aer.get_backend('aer_simulator')
        self.quantum_circuits = [VQCCircuit(kernel_size, backend) for _ in range(out_channels)]
        self.kernel_size = kernel_size

    def forward(self, x):
         # Aplica stratul cuantic personalizat pe input
        return QuanvFunction.apply(x, self.quantum_circuits, self.kernel_size)

# -------------------------- 4. Dataset Custom -----------------------------
MAX_SAMPLES = 9

class CTPatchDataset(Dataset):
      # Incarca imaginile din directoare structurate pe clase si le eticheteaza
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        label_map = {}
        label_id = 0

        for label_name in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, label_name)
            if not os.path.isdir(class_dir):
                continue

            key = label_name.strip().lower().replace(" ", "")
            if key not in label_map:
                label_map[key] = label_id
                label_id += 1

            label = label_map[key]

            for fname in os.listdir(class_dir):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    img_path = os.path.join(class_dir, fname)
                    self.samples.append((img_path, label))

        self.samples = self.samples[:MAX_SAMPLES]
        if len(self.samples) == 0:
            raise ValueError("Dataset-ul nu conține imagini valide.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('L')
        if self.transform:
            img = self.transform(img)
        return img, label

# -------------------------- 5. Modelul Hibrid ------------------------------
class HybridNet(nn.Module):
    def __init__(self):
        super(HybridNet, self).__init__()
        self.quanv = QuanvLayer(1, 1, kernel_size=3)
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)

        dummy = torch.zeros(1, 1, 28, 28)
        with torch.no_grad():
            x = self.forward_features(dummy)
            self.flatten_dim = x.view(1, -1).shape[1]

        self.fc1 = nn.Linear(self.flatten_dim, 64)
        self.fc2 = nn.Linear(64, 3)

    def forward_features(self, x):
        x = self.quanv(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# -------------------------- 6. Antrenare ------------------------------

# Defineste transformarile pentru imagini
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = CTPatchDataset(r"C:\Users\Luchi\Desktop\LicentaCompleta\Imagini\QNNTrain", transform=transform)
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 3
loss_history = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    total_batches = len(dataloader)
    print(f"\n Epoca {epoch+1}/{epochs} ({total_batches} batch-uri):")

    progress_bar = tqdm(enumerate(dataloader), total=total_batches, desc=f"Epoca {epoch+1}")
    for batch_idx, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = running_loss / total_batches
    loss_history.append(avg_loss)
    print(f" Epoca {epoch+1} finalizată. Loss mediu: {avg_loss:.4f}")

# -------------------------- 7. Evaluare ------------------------------
model_save_path = "hybrid_quantum_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f" Model salvat în '{model_save_path}'")

# Plot loss
plt.figure(figsize=(8,5))
plt.plot(range(1, epochs+1), loss_history, marker='o')
plt.title("Evoluția Loss-ului pe epoci")
plt.xlabel("Epoca")
plt.ylabel("Loss mediu")
plt.grid(True)
plt.show()

# Matrice confuzie
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Matricea de Confuzie")
plt.show()



 Epoca 1/3 (9 batch-uri):


Epoca 1:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
from qiskit.circuit import Parameter
from qiskit_aer import Aer
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.circuit.library import RawFeatureVector

from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# ----------------------- 1. VQC Cuantic cu parametri -----------------------
class VQCCircuit:
    def __init__(self, kernel_size, backend, shots=1024):
        self.n_qubits = kernel_size ** 2
        self.backend = backend
        self.shots = shots

        # Parametri trainabili ai circuitului
        self.params = np.random.uniform(0, 2 * np.pi, size=self.n_qubits * 2)

        # Circuit variational: rotații și entanglement
        self.vqc = TwoLocal(num_qubits=self.n_qubits,
                            rotation_blocks='ry',
                            entanglement_blocks='cz',
                            reps=1,
                            parameter_prefix='θ')

    def run(self, data):
        data = data.flatten()
        norm = np.linalg.norm(data)
        if norm != 0:
            data = data / norm

        # Codificare prin amplitudini
        feature_map = RawFeatureVector(self.n_qubits)
        qc = QuantumCircuit(self.n_qubits)
        qc.compose(feature_map.bind_parameters(data), inplace=True)
        qc.compose(self.vqc.bind_parameters(self.params), inplace=True)
        qc.measure_all()

        transpiled = transpile(qc, self.backend)
        job = self.backend.run(transpiled, shots=self.shots)
        result = job.result()
        counts = result.get_counts()

        total = sum(sum(int(bit) for bit in key) * val for key, val in counts.items())
        return total / (self.shots * self.n_qubits)

# -------------------------- 2. Funcția Forward -----------------------------
class QuanvFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, inputs, quantum_circuits, kernel_size):
        batch_size = inputs.shape[0]
        length_x = inputs.shape[2] - kernel_size + 1
        length_y = inputs.shape[3] - kernel_size + 1
        outputs = torch.zeros(batch_size, len(quantum_circuits), length_x, length_y).to(inputs.device)

        for i in range(batch_size):
            for c, circuit in enumerate(quantum_circuits):
                for x in range(length_x):
                    for y in range(length_y):
                        patch = inputs[i, 0, x:x+kernel_size, y:y+kernel_size]
                        outputs[i, c, x, y] = circuit.run(patch.cpu().detach().numpy())

        return outputs

    @staticmethod
    def backward(ctx, grad_output):
        return None, None, None

# ---------------------------- 3. Stratul Cuantic ----------------------------
class QuanvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(QuanvLayer, self).__init__()
        backend = Aer.get_backend('aer_simulator')
        self.quantum_circuits = [VQCCircuit(kernel_size, backend) for _ in range(out_channels)]
        self.kernel_size = kernel_size

    def forward(self, x):
        return QuanvFunction.apply(x, self.quantum_circuits, self.kernel_size)

# -------------------------- 4. Dataset Custom -----------------------------
MAX_SAMPLES = 30

class CTPatchDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        label_map = {}
        label_id = 0

        for label_name in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, label_name)
            if not os.path.isdir(class_dir):
                continue

            key = label_name.strip().lower().replace(" ", "")
            if key not in label_map:
                label_map[key] = label_id
                label_id += 1

            label = label_map[key]

            for fname in os.listdir(class_dir):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    img_path = os.path.join(class_dir, fname)
                    self.samples.append((img_path, label))

        self.samples = self.samples[:MAX_SAMPLES]
        if len(self.samples) == 0:
            raise ValueError("Dataset-ul nu conține imagini valide.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('L')
        if self.transform:
            img = self.transform(img)
        return img, label

# -------------------------- 5. Modelul Hibrid ------------------------------
class HybridNet(nn.Module):
    def __init__(self):
        super(HybridNet, self).__init__()
        self.quanv = QuanvLayer(1, 1, kernel_size=3)
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)

        dummy = torch.zeros(1, 1, 28, 28)
        with torch.no_grad():
            x = self.forward_features(dummy)
            self.flatten_dim = x.view(1, -1).shape[1]

        self.fc1 = nn.Linear(self.flatten_dim, 64)
        self.fc2 = nn.Linear(64, 3)

    def forward_features(self, x):
        x = self.quanv(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# -------------------------- 6. Antrenare ------------------------------
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = CTPatchDataset("/Imagini/QNNTrain", transform=transform)
batch_size = 2
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 5
loss_history = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    total_batches = len(dataloader)
    print(f"\n Epoch {epoch+1}/{epochs} ({total_batches} batch-uri):")

    progress_bar = tqdm(enumerate(dataloader), total=total_batches, desc=f"Epoca {epoch+1}")
    for batch_idx, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = running_loss / total_batches
    loss_history.append(avg_loss)
    print(f" Epoch {epoch+1} finalizat. Loss mediu: {avg_loss:.4f}")

# Salvare model
model_save_path = "hybrid_quantum_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f" Model salvat în '{model_save_path}'")

# Plot loss
plt.figure(figsize=(8,5))
plt.plot(range(1, epochs+1), loss_history, marker='o')
plt.title("Evoluția Loss-ului pe epoci")
plt.xlabel("Epoca")
plt.ylabel("Loss mediu")
plt.grid(True)
plt.show()

# Matrice confuzie
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Matricea de Confuzie")
plt.show()


In [1]:
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import seaborn as sns

# 1. Salvează modelul PyTorch
torch.save(model.state_dict(), "hybrid_model.pth")
print(" Model salvat ca hybrid_model.pth")

# 2. Evaluare pe datele de antrenare (sau test dacă ai)
model.eval()
all_preds = []
all_labels = []
all_losses = []
with torch.no_grad():
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        all_losses.append(loss.item())

        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 3. Afișare Confusion Matrix și Acuratețe
acc = accuracy_score(all_labels, all_preds)
cm = confusion_matrix(all_labels, all_preds)

print(f" Acuratețe: {acc*100:.2f}%")

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Benign', 'Malignant', 'Normal'])
disp.plot(cmap='Blues')
plt.title("Matrice de Confuzie")
plt.show()

# 4. Afișare loss
plt.figure(figsize=(6, 4))
plt.plot(all_losses, marker='o')
plt.title("Loss per Batch")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'model' is not defined

In [3]:
%%writefile hybrid_net.py

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit_aer import Aer

from tqdm import tqdm

# ----------------------------- 1. Circuit Cuantic -----------------------------
class QuanvCircuit:
    def __init__(self, kernel_size, backend, shots=10, threshold=0.5):
        self.n_qubits = kernel_size ** 2
        self.theta = [Parameter(f'theta{i}') for i in range(self.n_qubits)]
        self.circuit = QuantumCircuit(self.n_qubits)
        for i in range(self.n_qubits):
            self.circuit.rx(self.theta[i], i)
        self.circuit.barrier()
        self.circuit.h(range(self.n_qubits))
        self.circuit.measure_all()

        self.backend = backend
        self.shots = shots
        self.threshold = threshold

    def run(self, data):
        data = data.flatten()
        param_dict = {self.theta[i]: np.pi if data[i] > self.threshold else 0.0 for i in range(self.n_qubits)}
        bound_circuit = self.circuit.assign_parameters(param_dict, inplace=False)
        transpiled = transpile(bound_circuit, self.backend)
        job = self.backend.run(transpiled, shots=self.shots)
        result = job.result()
        counts = result.get_counts()
        total = sum(sum(int(bit) for bit in key) * val for key, val in counts.items())
        return total / (self.shots * self.n_qubits)

# -------------------------- 2. Funcție Forward Quanv --------------------------
class QuanvFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, inputs, quantum_circuits, kernel_size):
        batch_size = inputs.shape[0]
        length_x = inputs.shape[2] - kernel_size + 1
        length_y = inputs.shape[3] - kernel_size + 1
        outputs = torch.zeros(batch_size, len(quantum_circuits), length_x, length_y).to(inputs.device)

        for i in range(batch_size):
            for c, circuit in enumerate(quantum_circuits):
                for x in range(length_x):
                    for y in range(length_y):
                        patch = inputs[i, 0, x:x+kernel_size, y:y+kernel_size]
                        outputs[i, c, x, y] = circuit.run(patch.cpu().detach().numpy())

        return outputs

    @staticmethod
    def backward(ctx, grad_output):
        return None, None, None

# ---------------------------- 3. Stratul Cuantic ------------------------------
class QuanvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(QuanvLayer, self).__init__()
        backend = Aer.get_backend('aer_simulator')
        self.quantum_circuits = [QuanvCircuit(kernel_size, backend) for _ in range(out_channels)]
        self.kernel_size = kernel_size

    def forward(self, x):
        return QuanvFunction.apply(x, self.quantum_circuits, self.kernel_size)


class HybridNet(nn.Module):
    def __init__(self):
        super(HybridNet, self).__init__()
        self.quanv = QuanvLayer(1, 1, kernel_size=3)
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)

        # Init temporar pentru a determina automat dimensiunea FC
        self._dummy_input = torch.zeros(1, 1, 28, 28)  # same shape as your dataset images
        with torch.no_grad():
            x = self.forward_features(self._dummy_input)
            self.flatten_dim = x.view(1, -1).shape[1]

        self.fc1 = nn.Linear(self.flatten_dim, 64)
        self.fc2 = nn.Linear(64, 3)

    def forward_features(self, x):
        x = self.quanv(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)



Writing hybrid_net.py
